<a href="https://colab.research.google.com/github/alejandrocarmu/FIWARE-VRP/blob/main/VRP_ORS%2BVROOM_V1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VRP con Open Route Service y VROOM
#### ORS + VROOM

* https://github.com/GIScience/openrouteservice-py
* https://openrouteservice.org/
* https://openrouteservice-py.readthedocs.io/en/latest/
* https://github.com/VROOM-Project/vroom/blob/master/docs/API.md

### Instalación de librerías
Se instalan las librerías necesarias para poder hacer uso del notebook.

In [22]:
!pip install pandas
!pip install datetime
!pip install openrouteservice
!pip install folium

     |████████████████████████████████| 60 kB 3.1 MB/s 
     |████████████████████████████████| 251 kB 11.3 MB/s 


### Configurar la llave API de ORS

Para correr el notebook, es necesario usar una llave API de la plataforma de Open Route Service.

In [3]:
import pandas as pd
import datetime
import openrouteservice as ors

API_KEY = '5b3ce3597851110001cf62482d6296cdeb7c4441a0b91f42ea31b15a'
client = ors.Client(key=API_KEY)

### Crear el Depósito
El primer paso es crear un depósito y las ubicaciones de recogida. El depósito es la casa o base de los vehículos de recogida y las ubicaciones son los puntos de recogida. Estos lugares son determinados con los puntos de contenedores de basura que se tendrían en Fiware.

In [4]:
depot = {   'name': 'Relleno Sanitario La Pradera',
            'location': (6.5171946910919, -75.25502818729257)}

### Configurar el Número de Vehículos
El número de vehículos se configura como 3. Después de correr todo el notebook se puede cambiar esta cantidad para observar su impacto en las rutas generadas.

In [5]:
num_vehicles = 3

### Crear los contenedores
Se crea un diccionario con los datos de cada uno de los contenedores a recoger. Cuando se tengan los dispositivos instalados para los contenedores se buscaría tomar los datos directamente desde Orion con este formato.

In [6]:
contenedores = [
    { 
        'name': 'Plaza Botero',
        'location': (6.2524857541249315, -75.5686261812429),
        'level': 100,
        'weight': 60,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    },
    {
        'name': 'Universidad de Medellín',
        'location': (6.231704691813073, -75.61160925921747),
        'level': 75,
        'weight': 39,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    },
    {
        'name': 'Parque Juanes de la Paz',
        'location': (6.290384323934336, -75.56945687270888),
        'level': 90,
        'weight': 54,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    },
    {
        'name': 'Parque del Poblado',
        'location': (6.210362095508166, -75.57088108434691),
        'level': 75,
        'weight': 42,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    },
    {
        'name': 'Alcaldía de Medellín',
        'location': (6.2451496127526225, -75.57371337731854),
        'level': 75,
        'weight': 42,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    },
    {
        'name': 'Parque Explora',
        'location': (6.271208962002754, -75.56556245591827),
        'level': 90,
        'weight': 54,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    },
    {
        'name': 'Museo de Arte Moderno de Medellín',
        'location': (6.2240110064206, -75.57412303992716),
        'level': 50,
        'weight': 30,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    },
    {
        'name': 'Parque de Belén',
        'location': (6.232689716085326, -75.59665353624818),
        'level': 60,
        'weight': 36,
        'open_time': datetime.datetime(2022, 1, 17, 8, 0, 0),
        'close_time': datetime.datetime(2022, 1, 17, 20, 0, 0),
    }
]


### Graficar mapa con todas las ubicaciones y el depósito
Se crea una gráfica del mapa con open street maps para visualizar todas las ubicaciones de recogida y el depósito.

In [7]:
import folium as fm
from folium import Marker as mk

coordenadas_medellin = (6.25184, -75.56359)
coordenadas_girardota = (6.3792, -75.4453) # Permiten visualizar el mapa con un mejor zoom y en un punto central entre los contenedores de basura y el depósito

m = fm.Map(location=coordenadas_girardota, tiles='openstreetmap', zoom_start=12)

for node in contenedores:
    tooltip = fm.map.Tooltip("<h4><b>{}</b></p><p>Nivel: <b>{}</b></p><p>Peso: <b>{}</b></p>".format(node['name'], node['level'], node['weight']))
    ttdep = fm.map.Tooltip("<h4><b>{}</b></p>".format(depot['name']))
    mk(location=node['location'], tooltip=tooltip, icon=(fm.Icon(color='gray', icon='trash', prefix='fa'))).add_child(fm.Popup('{}'.format(node['name']))).add_to(m)
    

mk(location=depot['location'], tooltip=ttdep, icon= (fm.Icon(color='red', icon='industry', prefix='fa'))).add_child(fm.Popup('{}'.format(depot['name']))).add_to(m)
m

### Crear el Data Frame con la información necesaria para el problema


Se crea una función para filtrar los contenedores que van a entrar en el modelo, de acuerdo al nivel captura por cada contenedor. Para esta simulación se define que los contenedores por debajo de un 75% de la capacidad de carga no entran en el modelo.

In [8]:
threshold_level = 75 

def filter_containers(contenedores, level):     
    # Se filtran o seleccionan los contenedores que harían parte del modelo de acuerdo al threshold especificado          
    cont_filt = [c for c in contenedores if c['level'] >= level]
    
    return cont_filt

def pd_containers(contenedores):
    import pandas as pd
    # Se obtiene un dataframe con los contenedores filtrados que harían parte del modelo
    keys = []
    vals = []
    for data in contenedores:
        val = []
        for k,v in data.items():
            keys.append(k)
            val.append(v)
        vals.append(val)
    filtered_df = pd.DataFrame([v for v in vals], columns=list(dict.fromkeys(keys)))
    return filtered_df

contenedores_filtrados = filter_containers(contenedores, threshold_level)
contenedores_data = pd_containers(contenedores_filtrados)
contenedores_data

,name,location,level,weight,open_time,close_time
0,Plaza Botero,"(6.2524857541249315, -75.5686261812429)",100,60,2022-01-17 08:00:00,2022-01-17 20:00:00
1,Universidad de Medellín,"(6.231704691813073, -75.61160925921747)",75,39,2022-01-17 08:00:00,2022-01-17 20:00:00
2,Parque Juanes de la Paz,"(6.290384323934336, -75.56945687270888)",90,54,2022-01-17 08:00:00,2022-01-17 20:00:00
3,Parque del Poblado,"(6.210362095508166, -75.57088108434691)",75,42,2022-01-17 08:00:00,2022-01-17 20:00:00
4,Alcaldía de Medellín,"(6.2451496127526225, -75.57371337731854)",75,42,2022-01-17 08:00:00,2022-01-17 20:00:00
5,Parque Explora,"(6.271208962002754, -75.56556245591827)",90,54,2022-01-17 08:00:00,2022-01-17 20:00:00


Se crea un dataframe con los datos necesarios de los contenedores que serán utilizados posteriormente para la solicitud enviada a la API de optimización de ORS. Este dataframe servirá como fuente principal para identificar tanto las ubicaciones de los contenedores, los nombres de dichas ubicaciones, como los pesos de cada contenedor (que serían las cantidades solicitadas a recolectar).

In [9]:
contenedores_data['Lon'] = contenedores_data['location'].apply(lambda x: x[1])
contenedores_data['Lat'] = contenedores_data['location'].apply(lambda x: x[0])
contenedores_data

,name,location,level,weight,open_time,close_time,Lon,Lat
0,Plaza Botero,"(6.2524857541249315, -75.5686261812429)",100,60,2022-01-17 08:00:00,2022-01-17 20:00:00,-75.568626,6.252486
1,Universidad de Medellín,"(6.231704691813073, -75.61160925921747)",75,39,2022-01-17 08:00:00,2022-01-17 20:00:00,-75.611609,6.231705
2,Parque Juanes de la Paz,"(6.290384323934336, -75.56945687270888)",90,54,2022-01-17 08:00:00,2022-01-17 20:00:00,-75.569457,6.290384
3,Parque del Poblado,"(6.210362095508166, -75.57088108434691)",75,42,2022-01-17 08:00:00,2022-01-17 20:00:00,-75.570881,6.210362
4,Alcaldía de Medellín,"(6.2451496127526225, -75.57371337731854)",75,42,2022-01-17 08:00:00,2022-01-17 20:00:00,-75.573713,6.245150
5,Parque Explora,"(6.271208962002754, -75.56556245591827)",90,54,2022-01-17 08:00:00,2022-01-17 20:00:00,-75.565562,6.271209


### Crear los datos para las Restricciones de los Vehículos
Se crean los datos correspondientes a las restricciones de capacidad de los vehículos recolectores de basura. Se debe tener presente que las unidades de capacidad deben coincidir. Además se agrega una variable que contenga las ventanas de tiempo para los vehículos, es decir sus horas de operación.

In [10]:
vehicle_capacities = [100]
inicio_turno = int(datetime.datetime(2022, 1, 17, 8, 0, 0).timestamp())
final_turno = int(datetime.datetime(2022, 1, 17, 20, 0, 0).timestamp())
vehicle_time_windows = [inicio_turno, final_turno] # 8-20:00, expresados en POSIX timestamp
vehicle_time_windows

[1642406400, 1642449600]

### Graficar en el Mapa el depósito y las ubicaciones de recogida
En este paso solo se tienen en cuenta los contenedores filtrados anteriormente, así que se grafican los contenedores que superen el threshold determinado y el depósito de basura.

In [11]:
m1 = fm.Map(location=coordenadas_girardota, tiles='openstreetmap', zoom_start=12)

for node in contenedores_filtrados:
    tooltip = fm.map.Tooltip("<h4><b>{}</b></p><p>Nivel: <b>{}</b></p><p>Peso: <b>{}</b></p>".format(node['name'], node['level'], node['weight']))
    ttdep = fm.map.Tooltip("<h4><b>{}</b></p>".format(depot['name']))
    mk(location=node['location'], tooltip=tooltip, icon=(fm.Icon(color='gray', icon='trash', prefix='fa'))).add_child(fm.Popup('{}'.format(node['name']))).add_to(m1)
    

mk(location=depot['location'], tooltip=ttdep, icon= (fm.Icon(color='red', icon='industry', prefix='fa'))).add_child(fm.Popup('{}'.format(depot['name']))).add_to(m1)
m1

### Configurar el problema de Ruteo
En este paso se comienza a configurar el problema de el Vehicle Routing Problem. Para este modelo se usa la librería [Vroom](https://github.com/VROOM-Project/vroom), la cual tiene [soporte](http://k1z.blog.uni-heidelberg.de/2019/01/24/solve-routing-optimization-with-vroom-ors/) para Open Route Service y está disponible a través de APIs.

Para describir el problema adecuadamente en términos algorítmicos, se posee la siguiente información:

- **Dirección inicio/final de los vehículos**: el depósito de los vehículos es el Relleno Sanitario La Pradera.
- **Capacidad de los vehículos**: 100
- **Tiempos de operación de los vehículos**: 08:00 - 20:00
- **Ubicación de servicio**: Ubicación de recogida de los contenedores
- **Ventanas de tiempo de servicio**: Ventanas de tiempo de cada uno de los contenedores
- **Cantidad de servicio**: Cantidad a recoger en cada uno de los contenedores


Ahora se procede a organizar toda esta información y se envía la solicitud al servicio de optimización de Open Route Service en [`https://api.openrouteservice.org/optimization`](https://openrouteservice.org/dev/#/api-docs/optimization/post).

In [12]:
# Se definen los vehículos
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle

vehicles = list()
for vehicles_id in range(num_vehicles):
    vehicles.append(
        ors.optimization.Vehicle(
            id= vehicles_id,
            start= list(reversed(depot['location'])),
            end= list(reversed(depot['location'])),
            capacity= vehicle_capacities,
            time_window= vehicle_time_windows
        )
    )
    
# Luego se definen los puntos de recogida o los contenedores
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job

recogidas = list()
for recogida in contenedores_data.itertuples():
    recogidas.append(
        ors.optimization.Job(
            id=recogida.Index,
            location=[recogida.Lon, recogida.Lat],
            service=600,  # Se asumen 10 minutos en cada contenedor
            amount=[recogida.weight],
            time_windows=[[
                int(recogida.open_time.timestamp()),  # VROOM espera UNIX timestamp
                int(recogida.close_time.timestamp())
            ]]
        )
    )

### Hacer la solicitud a la API de Optimización
Ahora con la configuración realizada se procede a hacer la solicitud a la API de optimización, para que open route service calcule las rutas óptimas y defina la programación de ruta de cada vehículo para los contenedores a recoger.

In [13]:
# Se hace la solicitud

result = client.optimization(
    jobs=recogidas,
    vehicles=vehicles,
    geometry=True
)

### Verificar los resultados obtenidos
Ahora se procede a verificar los datos obtenidos de la API de optimización, para el problemas configurado anteriormente.

### Programa general de rutas


In [14]:
# Solo se extraen los campos relevantes de la respuesta

extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
vehicles_df

,distance,amount,duration
vehicle,,,
0,118301,[99],7142
1,118627,[96],7174
2,109566,[96],6326


Se procede a crear una lista para visualizar las rutas individuales.

In [15]:
# Se crea una lista para mostrar el programa para todos los vehículos

stations = list()
for route in result['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # ID de Contenedor
                step["arrival"],  # Tiempo de llegada
                step["arrival"] + step.get("service", 0),  # Tiempo de salida

            ]
        )
    stations.append(vehicle)

Se crea un dataframe para tener presentes los nombres de los contenedores de acuerdo a su ID.

In [16]:
nombres = pd.DataFrame(contenedores_data['name'])
nombres

,name
0,Plaza Botero
1,Universidad de Medellín
2,Parque Juanes de la Paz
3,Parque del Poblado
4,Alcaldía de Medellín
5,Parque Explora


Se crea una función para obtener el dataframe con las rutas para cada vehículo.

In [17]:
def df_stations(stations, num_vehicle):
    df_stations = pd.DataFrame(stations[num_vehicle], columns=["ID Contenedor", "Llegada", "Salida"])
    df_stations['Nombre'] = df_stations['ID Contenedor'].apply(lambda x: depot['name'] if x == 'Depot' else contenedores_data['name'][x])
    df_stations['Llegada'] = pd.to_datetime(df_stations['Llegada'], unit='s').dt.tz_localize(tz='UTC').dt.tz_convert('America/Bogota')
    df_stations['Salida'] = pd.to_datetime(df_stations['Salida'], unit='s').dt.tz_localize(tz='UTC').dt.tz_convert('America/Bogota')
    return df_stations  

Ahora se presenta la tabla individual de cada vehículo con sus tiempos de llegada y de salida a cada punto de recogida o contenedor, evidenciando la ruta a seguir, los tiempos de llegada y salida y el nombre del lugar.

### Vehículo 0

In [18]:
df_stations(stations, 0)

,ID Contenedor,Llegada,Salida,Nombre
0,Depot,2022-01-17 03:00:00-05:00,2022-01-17 03:00:00-05:00,Relleno Sanitario La Pradera
1,1,2022-01-17 03:50:44-05:00,2022-01-17 04:00:44-05:00,Universidad de Medellín
2,0,2022-01-17 04:13:31-05:00,2022-01-17 04:23:31-05:00,Plaza Botero
3,Depot,2022-01-17 05:19:02-05:00,2022-01-17 05:19:02-05:00,Relleno Sanitario La Pradera


### Vehículo1

In [19]:
df_stations(stations, 1)

,ID Contenedor,Llegada,Salida,Nombre
0,Depot,2022-01-17 03:00:00-05:00,2022-01-17 03:00:00-05:00,Relleno Sanitario La Pradera
1,3,2022-01-17 03:51:31-05:00,2022-01-17 04:01:31-05:00,Parque del Poblado
2,2,2022-01-17 04:17:42-05:00,2022-01-17 04:27:42-05:00,Parque Juanes de la Paz
3,Depot,2022-01-17 05:19:34-05:00,2022-01-17 05:19:34-05:00,Relleno Sanitario La Pradera


### Vehículo 2

In [20]:
df_stations(stations, 2)

,ID Contenedor,Llegada,Salida,Nombre
0,Depot,2022-01-17 03:00:00-05:00,2022-01-17 03:00:00-05:00,Relleno Sanitario La Pradera
1,4,2022-01-17 03:45:37-05:00,2022-01-17 03:55:37-05:00,Alcaldía de Medellín
2,5,2022-01-17 04:01:27-05:00,2022-01-17 04:11:27-05:00,Parque Explora
3,Depot,2022-01-17 05:05:26-05:00,2022-01-17 05:05:26-05:00,Relleno Sanitario La Pradera


### Graficar la solución en el mapa
Ahora se procede a graficar la solución en el mapa y a guardar el mapa en un archivo html, para poder interactuar y visualizarlo mejor.

In [21]:
# Agregar el resultado al mapa

for color, route in zip(['red','green','blue'], result['routes']):
    decoded = ors.convert.decode_polyline(route['geometry'])  # La geometría de la ruta está codificada
    gj = fm.GeoJson(
        name='Vehículo {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                         }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(fm.Tooltip(
        """<h4>Vehículo {vehicle}</h4>
        <b>Distancia:</b> {distance} m <br>
        <b>Duración:</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m1)

fm.LayerControl().add_to(m1)
m1.save('mapa_vrp_ors+vroom.html')
m1